In [13]:
import baostock as bs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, sys
from simulation import fetch_info, get_price, get_rsi, initialize, get_trading_dates, change_rate, simulation_start_date, next_date, analysis

In [14]:
stock_list = ['sh.600000', 'sh.600004', 'sh.600006']
start_date = '2019-01-01'
end_date = '2021-12-31'

In [15]:
def buy_signal(date, rsi6, rsi24):
    buy = 1
    # RSI24  < 30 and
    if rsi24[date] >= 0.3:
        buy = 0
    # RSI6 < 50 and
    if rsi6[date] >= 0.6:
        buy = 0
    # RSI6 - RSI24 > 0.5
    if rsi6[date] - rsi24[date] <= 0.1:
        buy = 0
    return buy

def sell_signal(buy_date, date, rsi6, rsi24, close_price):
    sell = 0
    # loss = 5% or
    if change_rate(buy_date, date, close_price) < -0.05:
        sell = 1
    # profit = 10% or
    if change_rate(buy_date, date, close_price) > 0.1:
        sell = 1
    # RSI6 > 75 or
    if rsi6[date] > 0.75:
        sell = 1
    # RSI24 > 60 or
    if rsi24[date] > 0.60:
        sell = 1
    # RSI6 - RSI24 < 0
    if rsi6[date] - rsi24[date] < 0.0:
        sell = 1
    return sell

In [16]:
# simulation
def simulate(stock_num, start_date, end_date, 
             close_price, trading_date_list, date,
             rsi6, rsi24):
    
    original = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    rsi6, rsi24 = get_rsi(stock_num, start_date, end_date)
    close_price = get_price(stock_num, start_date, end_date)
    trading_date_list = get_trading_dates(stock_num, start_date, end_date)
    sys.stdout = original
    position = 0 # 1 for stock, 0 for cash
    date = simulation_start_date(trading_date_list)
    record = []
    buy_date = None
    while True:
        if position == 0:
            if buy_signal(date, rsi6, rsi24) == 1:
                # all in
                position = 1
                buy_date = date
                # record behavior
                record.append((date, 'buy'))
            else:
                # stand
                pass
        else:
            if sell_signal(buy_date, date, rsi6, rsi24, close_price) == 1:
                # sell all
                position = 0
                # record behavior
                record.append((date, 'sell'))
            else:
                # stand
                pass
        if date == end_date:
            break
        # move to next trading day
        date = next_date(date, trading_date_list)
        
    return record

In [17]:
profit = []
for stock_num in stock_list:
    original = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    close_price, trading_date_list, date = initialize(stock_num, start_date, end_date)
    rsi6, rsi24 = get_rsi(stock_num, start_date, end_date)
    sys.stdout = original
    
    record = simulate(stock_num, start_date, end_date, 
                      close_price, trading_date_list, date,
                      rsi6, rsi24)
    average, accumulation = analysis(record, close_price, show = 0)
    profit.append([stock_num, average, accumulation])
df = pd.DataFrame(profit, columns = ['Stock num', 'Average profit', 'Accumulated profit'])
df

,Stock num,Average profit,Accumulated profit
0,sh.600000,0.008393,0.030828
1,sh.600004,-0.004205,-0.023632
2,sh.600006,-0.004007,-0.027128
